In [1]:
import pandas as pd
import data_work 
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline



In [2]:
import re


def load_data():
    df_1 = pd.read_csv('../data/customers.csv')
    df_2 = pd.read_csv('../data/contracts.csv')
    df_1.rename(columns = {'id':'customer_id'}, inplace=True)
    df_1.set_index('customer_id', inplace=True)
    df_2.set_index('customer_id', inplace=True)
    
    return df_2.join(df_1)

def clean_values(df):
    for each in ['deposit', 'monthly_amt']:
        df.loc[:, each] = pd.to_numeric([re.sub('[^0-9.]', '', str(s)) for s in df[each]])
    
    return df

def summary_calculations(df):
    df['monthly_total_revenue'] = df['monthly_amt'] * df['contract_term']
    df['total_revenue'] = df['deposit'] + df['monthly_total_revenue']
    
    return df

def expand_timeline(row):
    cu_id = row['customer_id']
    co_id = row['contract_id']
    st_dt = row['contract_start']
    m_term = row['contract_term']
    dep = row['deposit']
    mnthly = row['monthly_amt']
    
    new_data = []
    
    deposit_row = [
        cu_id, 
        co_id,
        st_dt,
        True, 
        dep
    ]
    new_data.append(deposit_row)

    for ea in range(m_term):
        monthly_row = [
            cu_id,
            co_id,
            (st_dt + timedelta(weeks=(ea * 4))),
            False,
            mnthly
        ]
        new_data.append(monthly_row)
        df = pd.DataFrame(new_data, columns = ['customer_id', 'contract_id', 'due_date', 'deposit', 'amt_due'])
    return df

def detail_calculations(orig_df):
    orig_df['contract_start'] = pd.to_datetime(orig_df['contract_start'], format='%m/%d/%Y')
    orig_df = orig_df.reset_index()
    interim_df = orig_df.apply(lambda x: expand_timeline(x), axis=1)
    
    new_df = pd.DataFrame()
    for each in range(len(interim_df)):
        new_df = new_df.append(interim_df[each])
      
    df_c = pd.read_csv('../data/customers.csv')
    df_c.rename(columns = {'id':'customer_id'}, inplace=True)
    df_c.set_index('customer_id', inplace=True)
    new_df.set_index('customer_id', inplace=True)
    new_df = new_df.join(df_c)
    new_df.reset_index(inplace=True)
    
    new_df.set_index('due_date', inplace=True)
    new_df.sort_index(inplace=True)
    
    return new_df

In [3]:
df = data_work.load_data()
df = data_work.clean_values(df)

summary = data_work.summary_calculations(df)

df_detail = data_work.detail_calculations(df)

df_time = df_detail.reset_index()
df_time['year'] = df_time['due_date'].dt.year
df_time['month'] = df_time['due_date'].dt.month

ValueError: time data '5/31/19' does not match format '%m/%d/%Y' (match)

In [ ]:
df_time

In [ ]:
df_detail.tail()

In [ ]:
df_detail['amt_due'].plot()

In [ ]:
df_detail.loc[df_detail['deposit'], 'amt_due'].plot()

In [ ]:
df_detail.loc[(
    (df_detail['deposit'] == False) &
    (df_detail['contract_id'] == 974)
), 'amt_due'].plot()

In [ ]:
df_detail.loc[(
    (df_detail['deposit'] == False) &
    (df_detail['industry'] == 'Television Services')
), 'amt_due'].plot()

In [ ]:
df_detail.groupby('state').agg({'amt_due':'sum'}).plot.bar(y = 'amt_due', x= 'state')

In [ ]:
df_detail.groupby('state', as_index=False).agg({'amt_due':'sum'}).plot.bar(y = 'amt_due', x= 'state')

In [ ]:
df_detail.groupby('state', as_index=False).agg({'amt_due':'sum'}).sort_values('amt_due').plot.bar(y = 'amt_due', x= 'state')

In [ ]:
df_v = df_detail.reset_index()
df_v['year'] = df_v['due_date'].dt.year
df_v['month'] = df_v['due_date'].dt.month
df_v.head()

In [ ]:
df_v.groupby(['year', 'month']).agg({'amt_due':sum}).plot()

In [ ]:
df_v.groupby(['year', 'month']).agg({'amt_due':sum}).plot(figsize=(20, 10))

In [ ]:
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)

df_v.groupby(['year', 'month']).agg({'amt_due':sum}).plot(figsize=(20, 10)).yaxis.set_major_formatter(tick)
plt.xticks(rotation=45)